# Many to One Classification

Simple example for Many to one (word sentiment classification) by Recurrent Neural Networks


- Creating the **data pipeline** with `tf.data`
- Preprocessing word sequences (variable input sequence length) using `padding technique` by `user function (pad_seq)`
- Using `tf.nn.embedding_lookup` for getting vector of tokens (eg. word, character)
- Creating the model as **Class**
- Reference
    - https://github.com/golbin/TensorFlow-Tutorials/blob/master/10%20-%20RNN/02%20-%20Autocomplete.py
    - https://github.com/aisolab/TF_code_examples_for_Deep_learning/blob/master/Tutorial%20of%20implementing%20Sequence%20classification%20with%20RNN%20series.ipynb


### Setup

In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import string
%matplotlib inline
slim = tf.contrib.slim

print(tf.__version__)

1.10.0


### Prepare example data

In [2]:
words = ['good', 'bad', 'amazing', 'so good', 'bull shit', 'awesome']
y = [[1.,0.], [0.,1.], [1.,0.], [1., 0.],[0.,1.], [1.,0.]]

In [3]:
# Character quantization
char_space = string.ascii_lowercase 
char_space = char_space + ' ' + '*'
char_space

'abcdefghijklmnopqrstuvwxyz *'

In [4]:
char_dic = {char : idx for idx, char in enumerate(char_space)}
print(char_dic)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25, ' ': 26, '*': 27}


### Create pad_seq function

In [5]:
def pad_seq(sequences, max_len, dic):
    seq_len, seq_indices = [], []
    for seq in sequences:
        seq_len.append(len(seq))
        seq_idx = [dic.get(char) for char in seq]
        seq_idx += (max_len - len(seq_idx)) * [dic.get('*')] # 27 is idx of meaningless token "*"
        seq_indices.append(seq_idx)
    return seq_len, seq_indices

### Apply pad_seq function to data

In [6]:
max_length = 10
X_length, X_indices = pad_seq(sequences = words, max_len = max_length, dic = char_dic)

In [7]:
print(X_length)
print(np.shape(X_indices))

[4, 3, 7, 7, 9, 7]
(6, 10)


### Define CharRNN class

In [ ]:
class CharRNN:
    def __init__(self, X_length, X_indices, y, n_of_classes, hidden_dim, dic):

        with tf.variable_scope('input_layer'):
            # input layer를 구현해보세요
            # tf.get_variable을 사용하세요
            # tf.nn.embedding_lookup을 사용하세요
            
            # 여기는 data pipeline
            self._X_length = X_length
            self._X_indices = X_indices
            self._y = y
            
            one_hot = tf.eye(len(dic), dtype = tf.float32)

        # RNN cell
        with tf.variable_scope('rnn_cell'):
            # RNN cell을 구현해보세요
            # tf.contrib.rnn.BasicRNNCell을 사용하세요
            # tf.nn.dynamic_rnn을 사용하세요
            pass
            
        with tf.variable_scope('output_layer'):
            # slim.fully_connected를 사용하세요
            pass

        with tf.variable_scope('loss'):
            # tf.losses.softmax_cross_entropy를 사용하세요
            pass
            
            
        with tf.variable_scope('prediction'):
            # tf.argmax를 사용하세요
            pass
        
    def predict(self, sess, X_length, X_indices):
        # predict instant method를 구현해보세요
        pass

### Create a model of CharGRU

In [8]:
# hyper-parameter#
lr = .003
epochs = 10
batch_size = 2
total_step = int(np.shape(X_indices)[0] / batch_size)
print(total_step)

3


In [ ]:
## create data pipeline with tf.data
# tf.data를 이용해서 직접구현해보세요

In [ ]:
# 최종적으로 model은 아래의 코드를 통해서 생성됩니다.
char_rnn = CharRNN(X_length = X_length_mb, X_indices = X_indices_mb, y = y_mb, n_of_classes = 2,
                   hidden_dim = 16, dic = char_dic)

### Creat training op and train model

In [ ]:
## create training op
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss = char_rnn.ce_loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

tr_loss_hist = []

for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0

    sess.run(tr_iterator.initializer)
    try:
        while True:
            # 여기를 직접구현하시면 됩니다.
            
    except tf.errors.OutOfRangeError:
        pass
    
    avg_tr_loss /= tr_step
    tr_loss_hist.append(avg_tr_loss)
    
    print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

In [ ]:
plt.plot(tr_loss_hist, label = 'train')

In [ ]:
yhat = char_rnn.predict(sess = sess, X_length = X_length, X_indices = X_indices)

In [ ]:
print('training acc: {:.2%}'.format(np.mean(yhat == np.argmax(y, axis = -1))))